In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
DATA_DIR = Path("data/processing_checkpoint")
path_clean_dataset = DATA_DIR / "01_cleaned_train.parquet"

In [3]:
df_clean_churn = pd.read_parquet(path_clean_dataset)

In [4]:
df_clean_churn.sample(5)

,gender,level,userId,page,sessionId,itemInSession,length,song,artist,time,registration,metropolitan_area,region,operating_system,browser
8845317,F,paid,1921726,NextSong,69608,207,212.50567,Live Like We're Dying,Kris Allen,2018-10-12 23:12:57,2018-09-14 14:45:42,Bowling Green,KY,Windows,Chrome
1818807,M,paid,1662255,Thumbs Down,112119,35,0.00000,No song,No artist,2018-10-23 05:50:20,2018-09-09 12:40:42,Portland-Vancouver-Hillsboro,OR-WA,Linux,Firefox
14047595,M,free,1130756,NextSong,5890,114,205.60934,Bulletproof,La Roux,2018-10-04 13:14:42,2018-05-14 04:18:23,Rockford,IL,Windows,Chrome
12841207,F,free,1968374,NextSong,110413,46,500.29669,Disposable Heroes (Album Version),Chimaira,2018-11-10 04:43:57,2018-09-03 03:42:38,Minneapolis-St. Paul-Bloomington,MN-WI,Macintosh,Firefox
3295635,F,paid,1113721,NextSong,83906,98,259.83955,Weed Hand,Baby Bash / Grimm / Lucky Luciano,2018-10-15 17:29:01,2018-09-24 17:18:13,Tampa-St. Petersburg-Clearwater,FL,Windows,Chrome


In [5]:
df_users_base = df_clean_churn[["userId", "gender", "registration", "operating_system", "browser"]].drop_duplicates(subset=["userId"])

In [6]:
len(df_users_base)

19140

In [7]:
df_users_base = df_users_base.set_index("userId")

In [8]:
df_unique_artists = (
    df_clean_churn.groupby("userId")["artist"].nunique().reset_index().rename(columns={"artist": "num_unique_artists"})
)
df_unique_artists = df_unique_artists.set_index("userId")
df_users_base = df_users_base.join(df_unique_artists)

In [9]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists
userId,,,,,
1773706,F,2018-09-18 20:30:10,Windows,Firefox,918
1538276,M,2018-08-15 07:40:43,Windows,Chrome,622
1277376,F,2018-09-05 16:50:56,Windows,Firefox,414
1608306,M,2018-08-21 12:06:39,Macintosh,Chrome,3337
1506159,F,2018-09-13 08:57:57,Windows,Firefox,1070


In [10]:
df_page_counts = (
    df_clean_churn.groupby("userId")["page"]
    .value_counts()
    .unstack(fill_value=0)
)

df_page_counts.columns = [f"count_{col.replace(' ', '_').lower()}" for col in df_page_counts.columns]

In [11]:
df_users_base = df_users_base.join(df_page_counts)
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_logout,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade
userId,,,,,,,,,,,,,,,,,,,,,
1584934,M,2018-08-28 06:41:45,Windows,Chrome,149,3,10,2,1,1,...,6,159,14,0,3,0,0,0,5,1
1560998,F,2018-09-05 13:21:40,Macintosh,Safari,1139,5,28,51,1,1,...,32,1632,30,1,8,1,2,16,69,2
1085157,M,2018-08-08 19:31:12,Macintosh,Safari,14,0,0,1,0,0,...,0,13,2,0,0,0,0,0,1,0
1583762,F,2018-09-20 15:27:40,Macintosh,Safari,1031,5,19,41,0,0,...,20,1421,2,1,9,0,1,15,65,1
1545408,F,2018-08-26 03:33:45,Macintosh,Chrome,37,0,0,2,0,0,...,2,36,0,0,0,0,0,1,3,0


In [12]:
df_session_count = (
    df_clean_churn.groupby("userId")["sessionId"]
    .nunique()
    .reset_index(name="count_total_sessions")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_session_count)


In [13]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions
userId,,,,,,,,,,,,,,,,,,,,,
1227865,F,2018-05-30 06:01:12,Macintosh,Firefox,248,3,6,9,0,0,...,268,13,1,3,0,1,8,11,2,7
1473508,M,2018-09-30 11:18:03,Windows,Firefox,766,0,14,34,0,0,...,996,7,1,7,0,1,9,44,1,7
1281757,M,2018-09-28 11:23:25,Windows,Chrome,33,0,0,0,0,0,...,32,5,0,1,0,0,1,0,0,3
1352736,F,2018-09-07 02:24:59,Macintosh,Chrome,403,0,4,13,1,1,...,469,29,0,2,0,1,5,21,6,8
1485572,F,2018-08-14 06:10:59,Windows,Firefox,131,0,5,7,0,0,...,139,17,0,2,0,0,0,7,2,5


In [14]:
# Build user lifecycle

df_last_time = (
    df_clean_churn.groupby("userId")["time"]
    .max()
    .reset_index(name="last_time")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_last_time)
df_users_base["user_lifecycle_h"] = df_users_base["last_time"] - df_users_base["registration"]
df_users_base["user_lifecycle_h"] = df_users_base["user_lifecycle_h"].dt.total_seconds() / 3600

df_users_base.head()

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,2,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,2,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,1,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,2,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889


In [15]:
df_length = (
    df_clean_churn.groupby("userId")["length"].sum()
)
df_length = (
    df_length
    .reset_index(name="ttl_length")
    .set_index("userId")
)
df_users_base = df_users_base.join(df_length)
df_users_base.head(5)


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168


In [16]:
df_item_per_session = (
    df_clean_churn.groupby("userId")["itemInSession"].max() / df_users_base["count_total_sessions"]
)

df_item_per_session = (
    df_item_per_session
    .reset_index(name="item_per_session")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_item_per_session)


In [17]:
df_users_base["frequency"] = df_users_base["count_total_sessions"] / df_users_base["user_lifecycle_h"]

In [18]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1291813,M,2018-09-14 15:49:49,Macintosh,Safari,1841,8,63,78,0,0,...,1,37,171,3,36,2018-11-18 14:42:32,1558.878611,736467.78834,13.583333,0.023094
1619079,F,2018-08-30 08:11:46,Macintosh,Safari,498,0,12,17,0,0,...,1,6,58,2,11,2018-11-19 15:19:25,1951.127500,149167.03527,19.181818,0.005638
1307123,M,2018-07-08 16:00:09,Windows,Chrome,280,0,8,7,0,0,...,1,2,7,3,9,2018-11-14 03:40:17,3083.668889,76647.27681,21.555556,0.002919
1181483,M,2018-06-21 12:48:13,Linux,Chrome,60,0,5,1,0,0,...,0,0,3,2,3,2018-11-07 22:35:16,3345.784167,14526.09150,12.333333,0.000897
1754536,F,2018-09-24 20:39:09,Windows,Firefox,74,1,0,1,0,0,...,0,2,3,0,4,2018-11-15 19:23:12,1246.734167,20836.30095,18.500000,0.003208


In [19]:
### df_users_base = df_users_base.drop(columns="user_lifecycle_h", errors="ignore")


In [20]:
"user_lifecycle_h" in df_users_base.columns

True

In [21]:
print(df_users_base["count_cancellation_confirmation"].unique())
df_users_base.rename(columns={"count_cancellation_confirmation":"churn_label"})

[1 0]


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,churn_label,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661,51.444444,0.005102
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861,29.666667,0.005822
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117,12.941176,0.019561
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024,25.650000,0.010984
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168,24.857143,0.009649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494594,M,2018-08-18 18:07:54,iPhone,Safari,12,0,0,0,0,0,...,0,0,0,0,1,2018-11-19 19:52:00,2233.735000,3047.43992,18.000000,0.000448
1036641,F,2018-07-09 12:04:12,Windows,Chrome,71,0,3,1,0,0,...,0,0,12,3,1,2018-11-19 23:58:21,3203.902500,17586.59262,102.000000,0.000312
1110980,F,2018-08-31 14:17:48,Windows,Chrome,51,0,0,4,0,0,...,0,0,5,0,1,2018-11-19 23:55:01,1929.620278,14253.34974,70.000000,0.000518


In [22]:
df_users_base["avg_songs_session"] = df_users_base["count_nextsong"] / df_users_base["count_total_sessions"]

In [23]:
mask = df_users_base["user_lifecycle_h"] < 24
df_users_base[mask]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.432500,15332.72528,79.0,0.095854,64.0
1444644,F,2018-10-06 04:01:43,Macintosh,Safari,40,0,0,2,0,0,...,0,0,0,1,2018-10-07 02:00:54,21.986389,9873.95455,46.0,0.045483,39.0
1039566,M,2018-10-07 10:56:31,iPhone,Safari,73,0,0,1,0,0,...,2,5,0,1,2018-10-07 16:32:59,5.607778,20749.91474,104.0,0.178324,75.0
1594905,M,2018-10-09 02:53:40,Windows,Chrome,304,2,2,11,0,0,...,4,15,0,1,2018-10-10 02:30:24,23.612222,84829.05169,399.0,0.042351,338.0
1745396,F,2018-11-07 15:38:32,Macintosh,Firefox,24,0,0,1,0,0,...,0,1,0,1,2018-11-07 17:02:16,1.395556,5221.47530,33.0,0.716561,23.0
1822231,M,2018-11-11 16:22:57,Windows,Firefox,11,0,1,1,0,0,...,0,1,0,1,2018-11-11 16:57:05,0.568889,2193.78487,28.0,1.757812,10.0
1792581,F,2018-11-12 12:06:34,Macintosh,Safari,17,0,0,1,0,0,...,0,0,0,1,2018-11-12 13:11:23,1.080278,4095.83557,38.0,0.925688,17.0
1094942,M,2018-11-13 18:48:19,Macintosh,Safari,140,1,0,4,0,0,...,0,7,0,1,2018-11-14 05:19:34,10.520833,38257.17415,193.0,0.095050,147.0
1351427,F,2018-11-14 10:14:32,Windows,Chrome,30,0,0,0,0,0,...,0,0,1,1,2018-11-14 12:37:59,2.390833,8783.92155,50.0,0.418264,29.0


In [24]:
df_users_base.loc[[1714714]]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.4325,15332.72528,79.0,0.095854,64.0


In [25]:
e = 0.0001
df_users_base["thumbs_ratio"] = df_users_base["count_thumbs_up"] / (df_users_base["count_thumbs_down"] + e)
df_users_base["ads_per_session"] = df_users_base["count_error"] / df_users_base["count_total_sessions"]
df_users_base["ads_per_session"] = df_users_base["count_roll_advert"] / df_users_base["count_total_sessions"]